In [1]:
import glob
import os
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime

In [3]:
# データ読み込み
import urbansound8k_loader as dataset

#parent_dir = 'UrbanSound8K/audio/'
#tr_features, tr_labels = dataset.load_urbansound8k(parent_dir, ['fold1', 'fold2'])
#ts_features, ts_labels = dataset.load_urbansound8k(parent_dir, ['fold3'])

%time tr_features, tr_labels, ts_features, ts_labels = dataset.load_from_npy_files('all_')

if (len(tr_features) !=  len(tr_labels)):
    print('WARN: invalid # of training data. features=' + str(len(tr_features)) + ', labels=' + str(len(tr_labels)))
elif (len(ts_features) !=  len(ts_labels)):
    print('WARN: invalid # of tast data. features=' + str(len(ts_features)) + ', labels=' + str(len(ts_labels)))
elif (len(tr_features) == 0 or len(ts_features) == 0):
    print('WARN: no data.')
else: 
    print('loaded successfully. # of train data=' + str(len(tr_features)) + ', # of test data=' + str(len(ts_features)))


CPU times: user 3.87 ms, sys: 23.5 ms, total: 27.3 ms
Wall time: 29.6 ms
loaded successfully. # of train data=6269, # of test data=2459


In [4]:
training_epochs = 5000
n_dim = tr_features.shape[1]  # 193
n_classes = 10
n_hidden_units_one = 280 
n_hidden_units_two = 300

In [ ]:
# モデル構築、学習、評価 by keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import keras.callbacks
import keras.backend.tensorflow_backend as KTF

log_filepath = './keras/log'
batch_size = 128
old_session = KTF.get_session()

with tf.Graph().as_default():
    session = tf.Session('')
    KTF.set_session(session)
    KTF.set_learning_phase(1)
    
    model = Sequential()
    model.add(Dense(n_hidden_units_one, activation='relu', input_shape=(n_dim,)))
    model.add(Dropout(0.2))
    model.add(Dense(n_hidden_units_two, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)
    cbks = [tb_cb]
    
    print ('fit start: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    history = model.fit(tr_features, tr_labels,
                        batch_size=batch_size,
                        epochs=training_epochs,
                        verbose=1,
                        callbacks=cbks, 
                        validation_data=(ts_features, ts_labels))

    print ('evaluate start: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    score = model.evaluate(ts_features, ts_labels, verbose=0)
    print('Test accuracy:', score[1])
    print ('finished: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

KTF.set_session(old_session)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 280)               54320     
_________________________________________________________________
dropout_1 (Dropout)          (None, 280)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               84300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3010      
Total params: 141,630
Trainable params: 141,630
Non-trainable params: 0
_________________________________________________________________
fit start:  2017/07/04 02:17:09
Train on 6269 samples, validate on 2459 samples
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal

6269/6269 [==============================] - 1s - loss: 0.1632 - acc: 0.9689 - val_loss: 3.9235 - val_acc: 0.5482
Epoch 53/5000
6269/6269 [==============================] - 1s - loss: 0.1756 - acc: 0.9654 - val_loss: 4.0022 - val_acc: 0.5519
Epoch 54/5000
6269/6269 [==============================] - 1s - loss: 0.1589 - acc: 0.9681 - val_loss: 3.9625 - val_acc: 0.5584
Epoch 55/5000
6269/6269 [==============================] - 1s - loss: 0.1701 - acc: 0.9679 - val_loss: 4.0308 - val_acc: 0.5478
Epoch 56/5000
6269/6269 [==============================] - 1s - loss: 0.1686 - acc: 0.9679 - val_loss: 4.0691 - val_acc: 0.5588
Epoch 57/5000
6269/6269 [==============================] - 1s - loss: 0.1587 - acc: 0.9727 - val_loss: 4.0486 - val_acc: 0.5575
Epoch 58/5000
6269/6269 [==============================] - 1s - loss: 0.1609 - acc: 0.9681 - val_loss: 4.3812 - val_acc: 0.5258
Epoch 59/5000
6269/6269 [==============================] - 1s - loss: 0.1567 - acc: 0.9713 - val_loss: 4.2385 - val_ac

6269/6269 [==============================] - 1s - loss: 0.1219 - acc: 0.9799 - val_loss: 4.8554 - val_acc: 0.5734
Epoch 116/5000
6269/6269 [==============================] - 1s - loss: 0.1384 - acc: 0.9789 - val_loss: 4.9078 - val_acc: 0.5539
Epoch 117/5000
6269/6269 [==============================] - 1s - loss: 0.1382 - acc: 0.9802 - val_loss: 4.7784 - val_acc: 0.5620
Epoch 118/5000
6269/6269 [==============================] - 1s - loss: 0.1300 - acc: 0.9783 - val_loss: 4.8472 - val_acc: 0.5608
Epoch 119/5000
6269/6269 [==============================] - 1s - loss: 0.1379 - acc: 0.9785 - val_loss: 4.6806 - val_acc: 0.5539
Epoch 120/5000
6269/6269 [==============================] - 1s - loss: 0.1371 - acc: 0.9778 - val_loss: 4.6677 - val_acc: 0.5539
Epoch 121/5000
6269/6269 [==============================] - 1s - loss: 0.1553 - acc: 0.9796 - val_loss: 5.0353 - val_acc: 0.5559
Epoch 122/5000
6269/6269 [==============================] - 1s - loss: 0.1438 - acc: 0.9793 - val_loss: 4.9833 -

6269/6269 [==============================] - 1s - loss: 0.1458 - acc: 0.9801 - val_loss: 5.4164 - val_acc: 0.5380
Epoch 179/5000
6269/6269 [==============================] - 1s - loss: 0.1195 - acc: 0.9813 - val_loss: 5.4836 - val_acc: 0.5490
Epoch 180/5000
6269/6269 [==============================] - 1s - loss: 0.1239 - acc: 0.9818 - val_loss: 5.2761 - val_acc: 0.5417
Epoch 181/5000
6269/6269 [==============================] - 1s - loss: 0.1243 - acc: 0.9815 - val_loss: 5.2892 - val_acc: 0.5486
Epoch 182/5000
6269/6269 [==============================] - 1s - loss: 0.1301 - acc: 0.9821 - val_loss: 5.2174 - val_acc: 0.5584
Epoch 183/5000
6269/6269 [==============================] - 1s - loss: 0.1474 - acc: 0.9807 - val_loss: 4.9768 - val_acc: 0.5584
Epoch 184/5000
6269/6269 [==============================] - 1s - loss: 0.1587 - acc: 0.9794 - val_loss: 5.0583 - val_acc: 0.5681
Epoch 185/5000
6269/6269 [==============================] - 1s - loss: 0.1263 - acc: 0.9828 - val_loss: 5.1681 -

6269/6269 [==============================] - 1s - loss: 0.1380 - acc: 0.9825 - val_loss: 5.2505 - val_acc: 0.5702
Epoch 242/5000
6269/6269 [==============================] - 1s - loss: 0.1367 - acc: 0.9833 - val_loss: 5.4436 - val_acc: 0.5641
Epoch 243/5000
6269/6269 [==============================] - 1s - loss: 0.1455 - acc: 0.9823 - val_loss: 5.4020 - val_acc: 0.5657
Epoch 244/5000
6269/6269 [==============================] - 1s - loss: 0.1454 - acc: 0.9812 - val_loss: 5.3195 - val_acc: 0.5514
Epoch 245/5000
6269/6269 [==============================] - 1s - loss: 0.1563 - acc: 0.9817 - val_loss: 5.3610 - val_acc: 0.5714
Epoch 246/5000
6269/6269 [==============================] - 1s - loss: 0.1404 - acc: 0.9829 - val_loss: 5.4263 - val_acc: 0.5462
Epoch 247/5000
6269/6269 [==============================] - 1s - loss: 0.1212 - acc: 0.9844 - val_loss: 5.5010 - val_acc: 0.5409
Epoch 248/5000
6269/6269 [==============================] - 1s - loss: 0.1235 - acc: 0.9836 - val_loss: 5.5882 -

6269/6269 [==============================] - 1s - loss: 0.1434 - acc: 0.9831 - val_loss: 5.7900 - val_acc: 0.5405
Epoch 305/5000
6269/6269 [==============================] - 1s - loss: 0.1384 - acc: 0.9836 - val_loss: 5.6501 - val_acc: 0.5555
Epoch 306/5000
6269/6269 [==============================] - 1s - loss: 0.1245 - acc: 0.9844 - val_loss: 5.4805 - val_acc: 0.5689
Epoch 307/5000
6269/6269 [==============================] - 1s - loss: 0.1313 - acc: 0.9855 - val_loss: 5.4957 - val_acc: 0.5645
Epoch 308/5000
6269/6269 [==============================] - 1s - loss: 0.1436 - acc: 0.9821 - val_loss: 5.3169 - val_acc: 0.5726
Epoch 309/5000
6269/6269 [==============================] - 1s - loss: 0.1417 - acc: 0.9831 - val_loss: 5.5014 - val_acc: 0.5523
Epoch 310/5000
6269/6269 [==============================] - 1s - loss: 0.1352 - acc: 0.9815 - val_loss: 5.6205 - val_acc: 0.5551
Epoch 311/5000
6269/6269 [==============================] - 1s - loss: 0.1416 - acc: 0.9829 - val_loss: 5.3638 -